# 이상치 나타내는 코드

In [19]:
import pandas as pd
import os

# 폴더 경로 지정 (사용자가 원하는 폴더 경로로 변경)
folder_path = "./dataset/VNSGN"

# 폴더 내 모든 CSV 파일 가져오기
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 모든 CSV 파일에 대해 반복 처리
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    
    # 위도 및 경도 차이 계산
    df["LAT_diff"] = df["LAT"].diff().abs()  # 이전 행과의 위도 차이
    df["LON_diff"] = df["LON"].diff().abs()  # 이전 행과의 경도 차이

    df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"])
    df["TIME_DIFF"] = df["TIMESTAMP"].diff() 

    # 이상치 조건 (경도 또는 위도가 1도 이상 차이)
    outlier_indices = df[(df["LAT_diff"] >= 1) | (df["LON_diff"] >= 1)].index

    # 결과 출력
    if not outlier_indices.empty:
        print(f"\n=== 파일명: {file} ===")
        for idx in outlier_indices:
            print(f"\n--- 이상치 데이터 (Index: {idx}) ---")
            print(df.loc[idx, ['TIMESTAMP','TIME_DIFF', 'SPEED', 'LAT', 'LON', 'LAT_diff', 'LON_diff']])  # 현재 이상치 데이터 출력
            
            # 앞 데이터 출력 (이전 행이 존재할 경우)
            if idx > 0:
                print(f"\n이전 데이터 (Index: {idx - 1}):")
                print(df.loc[idx - 1, ['TIMESTAMP','TIME_DIFF', 'SPEED', 'LAT', 'LON', 'LAT_diff', 'LON_diff']])

            print("=" * 50)  # 구분선 추가


=== 파일명: 4d3f1384-486b-3514-a34c-fb4d043741f7.csv ===

--- 이상치 데이터 (Index: 422) ---
TIMESTAMP    2024-07-21 01:57:05
TIME_DIFF        0 days 07:36:44
SPEED                       14.4
LAT                    18.551873
LON                   114.822848
LAT_diff                1.506298
LON_diff                1.124348
Name: 422, dtype: object

이전 데이터 (Index: 421):
TIMESTAMP    2024-07-20 18:20:21
TIME_DIFF        0 days 00:22:12
SPEED                       14.7
LAT                    20.058172
LON                   115.947197
LAT_diff                0.079938
LON_diff                  0.0444
Name: 421, dtype: object

--- 이상치 데이터 (Index: 423) ---
TIMESTAMP    2024-07-21 07:31:01
TIME_DIFF        0 days 05:33:56
SPEED                       12.9
LAT                    17.546833
LON                   114.015207
LAT_diff                 1.00504
LON_diff                0.807642
Name: 423, dtype: object

이전 데이터 (Index: 422):
TIMESTAMP    2024-07-21 01:57:05
TIME_DIFF        0 days 07:36:44
SPEED  

# 지도를 볼 수 있는 코드(폴더랑 파일명 지정)

In [ ]:
import pandas as pd
import folium

# CSV 파일 로드
file_path = "./dataset/JPKNZ/af2ff49a-4e82-3b27-af7d-b8ad38ffcd51.csv"  # 업로드한 파일 경로를 입력하세요.
df = pd.read_csv(file_path)

# 위도와 경도 데이터 추출
latitude = df["LAT"].tolist()
longitude = df["LON"].tolist()

# 지도 초기화 (첫 번째 위치 기준)
m = folium.Map(location=[latitude[0], longitude[0]], zoom_start=10)

# 마커 추가 (각 위치별)
for lat, lon, timestamp in zip(latitude, longitude, df["TIMESTAMP"]):
    folium.Marker(
        location=[lat, lon],
        popup=f"Time: {timestamp}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# 경로 표시 (폴리라인 추가)
folium.PolyLine(
    locations=list(zip(latitude, longitude)),
    color="blue",
    weight=4,
    opacity=0.7
).add_to(m)

# HTML 파일로 저장
m.save("map.html")

# 생성된 지도 파일 열기 (Jupyter Notebook 환경)
import webbrowser
webbrowser.open("map.html")

In [23]:
!pip install geopy

In [25]:
from geopy.distance import geodesic

def calculate_actual_distance(lat1, lon1, lat2, lon2):
    """
    두 좌표 (위도, 경도) 간의 실제 거리(Geodesic Distance)를 계산하는 함수.
    :param lat1: 첫 번째 위도
    :param lon1: 첫 번째 경도
    :param lat2: 두 번째 위도
    :param lon2: 두 번째 경도
    :return: 두 지점 간의 실제 거리 (km)
    """
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    return geodesic(point1, point2).kilometers

# 예제 사용법
lat1, lon1 = 36.428835, 134.500110
lat2, lon2 = 37.084632, 135.945963
actual_distance = calculate_actual_distance(lat1, lon1, lat2, lon2)

print(f"실제 이동 거리: {actual_distance:.2f} km")


실제 이동 거리: 148.20 km


# 속도를 이용한 예상 이동 거리 계산

In [27]:
def calculate_expected_distance(speed_knots, time_hours):
    """
    속도(knots)와 이동 시간을 이용하여 예상 이동 거리(km)를 계산하는 함수.
    :param speed_knots: 선박 속도 (노트 단위)
    :param time_hours: 이동 시간 (시간 단위)
    :return: 예상 이동 거리 (km)
    """
    speed_kmh = speed_knots * 1.852  # 노트를 km/h로 변환
    return speed_kmh * time_hours

# 예제 사용법
speed_knots = 12.45  # 평균 속도 (노트)
time_hours = 6.35  # 시간 차이 (6시간 21분 -> 6.35시간)
expected_distance = calculate_expected_distance(speed_knots, time_hours)

print(f"속도를 이용한 예상 이동 거리: {expected_distance:.2f} km")


속도를 이용한 예상 이동 거리: 146.41 km
